In [1]:
import tensorflow as tf
import CapsLayer as cp
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os

In [2]:
#read and resize from coco
shape=[64,64,3]
def gen_data(batch_size,shape=[256,256,3],path='data/unlabeled2017/'):
    num_of_digits = 12
    
    out = np.zeros([batch_size,shape[0],shape[1],shape[2]])
    b_num = 0
    idx = 1
    while True:
        f = "0"*(12-len(str(idx)))+str(idx)+".jpg"
        if b_num < batch_size:
            img = cv2.imread(path+f)
           # print(np.array(img[0]).shape)
            if(np.array(img).shape==()):
                if idx > 123392 // batch_size:
                    break
                idx+=1
                continue
        
            out[b_num] =  cv2.resize(img,(shape[0],shape[1]))
            
            b_num+=1
        else:
            b_num=0
            yield out
        
        idx+=1
        if idx > 123392 // batch_size:
            break

def gen_data(batch_size,shape=[256,256,3],path='data/unlabeled2017/'):
    out = np.zeros([batch_size,shape[0],shape[1],shape[2]])
    b_num = 0
    
    for f in os.listdir(path):
        if b_num < batch_size:
            
            out[b_num] = cv2.resize(cv2.imread(path+f),(shape[0],shape[1]))
            b_num+=1
        else:
            b_num = 0
            yield out
    
#data = gen_data(5,shape=shape)        


In [3]:
tf.reset_default_graph()

X = tf.placeholder(shape=[None, shape[0],shape[1], shape[2]], dtype=tf.float32, name="x_input")
Y = tf.placeholder(shape=[None, shape[0],shape[1], shape[2]], dtype=tf.float32, name="y_output")

In [4]:
#encoder
e_conv1 = tf.layers.conv2d(X, name="e_conv1", filters = 32, 
                             kernel_size = 9,strides = 2,padding = "valid", activation = tf.nn.relu)
e_conv2 = cp.conv_to_caps(e_conv1,num_maps=32,caps_dims=1,kernel_size = 9,strides = 2
                        ,name="e_conv2", activation = tf.nn.relu)
caps1 = cp.capsule(e_conv2,num_caps=1200,caps_dims=2,batch_size=tf.shape(X)[0],name="e_caps1")

caps_bridge = tf.reshape(caps1,[tf.shape(caps1)[0],caps1.shape[1],caps1.shape[2],1])


In [5]:
print(e_conv1)
print(e_conv2,caps1)

Tensor("e_conv1/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("e_conv2reshape:0", shape=(?, 3200, 1), dtype=float32) Tensor("e_caps1/Squeeze:0", shape=(?, 1200, 1), dtype=float32)


In [6]:
#decoder
d_conv1 = tf.layers.conv2d(caps_bridge, name="d_conv1", filters = 16, 
                             kernel_size = 1,strides = 1,padding = "valid", activation = tf.nn.relu)
d_conv2 = tf.layers.conv2d(d_conv1, name="d_conv2", filters = 32, 
                             kernel_size = 1,strides = 1,padding = "valid", activation = tf.nn.relu)
d_conv3 = tf.layers.conv2d(d_conv2, name="d_conv3", filters = 16, 
                             kernel_size = 1,strides = 1,padding = "valid", activation = tf.nn.relu)
d_conv4 = tf.layers.conv2d(d_conv3, name="d_conv4", filters = 32, 
                             kernel_size = 1,strides = 1,padding = "valid", activation = tf.nn.relu)
d_conv5 = tf.layers.conv2d(d_conv4, name="d_conv5", filters = 16, 
                             kernel_size = 1,strides = 1,padding = "valid", activation = tf.nn.relu)
d_conv5_flat = tf.reshape(d_conv5,[tf.shape(d_conv5)[0],d_conv5.shape[1]*d_conv5.shape[2]*d_conv5.shape[3]])

dense = tf.layers.dense(d_conv5_flat,shape[0]*shape[1]*shape[2])
output = tf.reshape(dense,[tf.shape(d_conv5_flat)[0],shape[0],shape[1],shape[2]])
merged = tf.summary.merge_all()
saver = tf.train.Saver()

In [ ]:
loss = tf.reduce_mean(tf.square(output-Y))

#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = output, labels = Y)) 

optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
training_op = optimizer.minimize(loss, name="training_op")
        
       
init = tf.global_variables_initializer()

loss_sum = tf.summary.scalar('Loss', loss)
output_image_sum = tf.summary.image('Generated_Image',output)
input_image_sum = tf.summary.image('Input_Image',X)
merged = tf.summary.merge_all()
saver = tf.train.Saver()

In [3]:
!rm -r AutoEncoder_logs/train


rm: cannot remove 'AutoEncoder_logs/train': No such file or directory


In [ ]:
n_epochs = 50
batch_size = 5
n_iterations_per_epoch = 123392 // batch_size



checkpoint_path = "./AutoEncoder_logs/caps_AE"
restore_checkpoint = False
save = False
tensorboard = False

print("Loading Graph")
sess = tf.Session()
if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
    saver.restore(sess, checkpoint_path)
else:
    sess.run(init)
    
train_writer = tf.summary.FileWriter('AutoEncoder_logs/train',
                                      sess.graph)

print("Starting Training")

step = 0
for epoch in range(n_epochs):
    for iteration,(batch) in enumerate(gen_data(batch_size,shape=shape)):
        step+=1
        loss_train = 0
        _,loss_train,pred,summary = sess.run([training_op,loss,output,merged],feed_dict={X: batch,
                       Y: batch})
        train_writer.add_summary(summary, step)
        print(("\rEpoch: {}  Iteration: {}/{} ({:.1f}%)  Loss: {:.5f}").format(
                      epoch+1,iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train),
                  end="")
        if save and iteration%100 == 0:
            save_path = saver.save(sess, checkpoint_path)
print("done")

Loading Graph
Starting Training
Epoch: 1  Iteration: 162/24678 (0.7%)  Loss: 5137.17529